In [128]:
scene_ = ["ESC:", "SEC:", "ESCENA:", "ESCENA", "SECUENCIA:", "SECUENCIA"]
in_out_ = ["INT./EXT", "EXT./INT","INT", "EXT"]
moment_ = ["TARDE-NOCHE","DÍA","UN RATO DESPUÉS" ,"DIA", "AMANECER", "MAÑANA", "TARDE", "ATARDECER", "ANOCHECER", "NOCHE", "MADRUGADA", "OCASO", "MÁS TARDE", "ALBA", "TARDE/OCASO", "AL DÍA SIGUIENTE", "MOMENTOS DESPUÉS", "CONTINÚA"]

In [129]:
scene_options = sorted(scene_, key=len, reverse=True)
in_out_options = sorted(in_out_, key=len, reverse=True)
moment_options = sorted(moment_, key=len, reverse=True)

In [130]:
Scene_options = '|'.join(scene_options)
In_out_options = '|'.join(in_out_options)
Moment_options = '|'.join(moment_options)

In [131]:
from sly import Lexer, Parser


class MyLexer(Lexer):
    tokens={SEPARADOR, SCENE_OPTIONS, IN_OUT_OPTIONS, MOMENT_OPTIONS, NUMBER,TEXT}

    ignore=' \t\n'

    SCENE_OPTIONS=rf'{Scene_options}'
    IN_OUT_OPTIONS = rf'{In_out_options}'
    MOMENT_OPTIONS = rf'{Moment_options}'
    NUMBER = r'\d+'
    SEPARADOR = r'\-|\-\-|\.'
    TEXT=r'[a-zA-Z0-9!@#$%^&*()_+=\[\]{};:\'",.<>/?\\|`~\-]+'

class MyParser(Parser):
    tokens = MyLexer.tokens

    # Regla principal de la gramática que reconoce la estructura general
    @_('opcional_escena opcional_numero in_out_opcion texto_con_separadores moment_options')
    def entrada(self, p):
        return ('entrada', p.opcional_escena, p.opcional_numero, p.in_out_opcion, p.texto_con_separadores, p.moment_options)

    # Reglas para manejar la opción de tener o no SCENE_OPTIONS
    @_('SCENE_OPTIONS')
    def opcional_escena(self, p):
        return p.SCENE_OPTIONS

    @_('empty')
    def opcional_escena(self, p):
        return None

    # Reglas para manejar la opción de tener o no NUMBER
    @_('NUMBER SEPARADOR')
    def opcional_numero(self, p):
        return p.NUMBER

    @_('empty')
    def opcional_numero(self, p):
        return None

    # Regla para manejar IN_OUT_OPTIONS seguido de un separador
    @_('IN_OUT_OPTIONS SEPARADOR')
    def in_out_opcion(self, p):
        return p.IN_OUT_OPTIONS

    @_('TEXT secuencia_de_texto')
    def texto_con_separadores(self, p):
        return f"{p.TEXT} {p.secuencia_de_texto}"
    
    @_('SEPARADOR TEXT secuencia_de_texto ')
    def secuencia_de_texto(self, p):
        return f"{p.SEPARADOR} {p.TEXT} {p.secuencia_de_texto}"
    
    @_('TEXT secuencia_de_texto')
    def secuencia_de_texto(self, p):
        return f"{p.TEXT} {p.secuencia_de_texto}"

    @_('TEXT SEPARADOR')
    def secuencia_de_texto(self, p):
        return p.TEXT
    
    @_('SEPARADOR')
    def secuencia_de_texto(self, p):
        return ''
    
    @_('MOMENT_OPTIONS')
    def moment_options(self, p):
        return p.MOMENT_OPTIONS

    # Regla para manejar casos vacíos (ausencia de SCENE_OPTIONS o NUMBER)
    @_('')
    def empty(self, p):
        return None

    # Manejo de errores sintácticos
    def error(self, p):
        if p:
            print(f"Error de sintaxis en el token {p.type}, valor {p.value}")
        else:
            print("Error de sintaxis en el final de la entrada")



In [133]:
from cmp.utils import *

lexer = MyLexer()
parser = MyParser()


examples = [
        'ESC: 20. INT./EXT. CASA ABUELA. PORTAL - UN RATO DESPUÉS'
    ]

for data in examples:
        token=lexer.tokenize(data)
        # for tok in token:
        #         print(tok)
        result = parser.parse(token)
        print(result)

('entrada', 'ESC:', '20', 'INT./EXT', 'CASA ABUELA. PORTAL', 'UN RATO DESPUÉS')


In [135]:
from docx2pdf import convert

convert ( input_path="/home/dario/Study/mma/Abuelas_y_MazmorrasV3.1.docx")

/home/dario/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NotImplementedError: docx2pdf is not implemented for linux as it requires Microsoft Word to be installed